In [ ]:
import pymysql
import pandas as pd
img=pd.read_csv('transformation.csv')
img

In [ ]:
id_list=[]
img_id=img['id']

for i in set(img_id):
    id_list.append(i)
id_list

In [ ]:
try:
    connection=pymysql.Connect(host='localhost',user='root',passwd='',database='colourme',cursorclass=pymysql.cursors.DictCursor)
    for id in id_list:
        mask=(img['id']==id)
        img_select=img[mask]
        for i in range(len(img_select.columns)):
            img_select=img_select.rename(columns={str(i):f"px{i}"})
        img_select=img_select.drop(['id'],axis=1)
        imgr=img_select.T
        print('表格處理完成')
    
        cols=[]
        for i in range(len(imgr.columns)):
            cols.append(f'pic{i+1}')
        cols=str(cols).replace('[','').replace(']','').replace("'","`")
    
        pid=f'p{id}'
        with connection.cursor() as cursor:
            command_table=f'CREATE TABLE IF NOT EXISTS {pid}(title CHAR(10) NOT NULL)'
            cursor.execute(command_table)
            print(command_table)
            for i in range(len(imgr.columns)):
                command_add_colums=f"ALTER TABLE {pid} ADD pic{i+1} int(10) NOT NULL"
                cursor.execute(command_add_colums)
                connection.commit()

    
        for k in range(len(imgr.index)):
            px=[]
            for i in imgr.loc[f'px{k}']:
                    px.append(i)
            px=str(px).replace("[","'").replace("]","'").replace(", ","', '")
            with connection.cursor() as cursor:
                    command_insert=f"INSERT INTO `{pid}` (`title`,{cols}) VALUES('3333',{px})"
                    cursor.execute(command_insert)
                    connection.commit()
        print(f'{id}完成')
        
except Exception as ex:
    print(ex)